In [1]:
import os
os.environ["SSL_CERT_FILE"] = "Fortinet_CA_SSL(15).cer"


In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[19]

{'text': 'Yes, for simplicity (of troubleshooting against the recorded videos) and stability. [source]\nBut Python 3.10 and 3.11 should work fine.',
 'section': 'General course-related questions',
 'question': 'Environment - Is Python 3.9 still the recommended version to use in 2024?',
 'course': 'data-engineering-zoomcamp'}

In [4]:
# # create an id
# # this isn't the best idea as when update the file that will change the ids 

# n = len(documents)

# for i in range(n):
#     documents[i]["id"] = i

In [5]:
documents[19]

{'text': 'Yes, for simplicity (of troubleshooting against the recorded videos) and stability. [source]\nBut Python 3.10 and 3.11 should work fine.',
 'section': 'General course-related questions',
 'question': 'Environment - Is Python 3.9 still the recommended version to use in 2024?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
# The issue of this method if someone changed the text that will change the id too
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [7]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [8]:
from collections import defaultdict

In [9]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [10]:
len(hashes), len(documents)
# they should produce the same length 

(947, 948)

In [11]:
# to get the dublication id 
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

593f7569 2


In [12]:
hashes['593f7569']

#they start with the same text 

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'}]

In [13]:
import json

In [14]:
# save the output to json 

with open('documents_with_ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [15]:
!head documents_with_ids.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "id": "c02e79ef"
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


### Prepare user questions

In [16]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks like:

["question1", "question2", ..., "question5"]
""".strip()

In [17]:
from mistralai import Mistral
from mistralai.models import UserMessage

In [18]:
from dotenv import load_dotenv

In [19]:
load_dotenv()

True

In [20]:
api_key = os.getenv("API_KEY")

In [21]:
client = Mistral(api_key = api_key, )

In [22]:
doc = documents[2]
prompt = prompt_template.format(**doc)

In [23]:
print(prompt)

You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Provide the output in parsable JSON without using code blocks like:

["question1", "question2", ..., "question5"]


In [24]:
def extract_json(text):
    """
    Extracts the JSON string from a string that may include ```json ``` markers.

    Args:
        text: The input string.

    Returns:
        The extracted JSON string, or None if no JSON is found.
    """
    import re
    match = re.search(r"```json\s*([\s\S]*?)\s*```", text)
    if match:
        return match.group(1)
    else:
        return None

text_with_json = "Here is some text. ```json {\"key\": \"value\"} ``` And some more text."
json_string = extract_json(text_with_json)

if json_string:
    print(f"Extracted JSON: {json_string}")
else:
    print("No JSON found in the text.")

Extracted JSON: {"key": "value"}


In [30]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)
    
    response = client.chat.complete(
        model = "open-mistral-7b", 
        messages = [UserMessage(content=prompt)]
    )
    json_response = extract_json(response.choices[0].message.content)
    return json_response

In [31]:
from tqdm.auto import tqdm

In [32]:
results = {}

for doc in tqdm(documents):
    doc_id = doc['id']
    if doc_id in results:
        continue 
        
    questions = generate_questions(doc)
    results[doc_id] = questions

  0%|          | 0/948 [00:00<?, ?it/s]

In [40]:
import pickle

In [41]:
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)


In [39]:
results['1f6520ca']

In [36]:
parsed_resulst = {}

for doc_id, json_questions in results.items():
    parsed_resulst[doc_id] = json.loads(json_questions)

TypeError: the JSON object must be str, bytes or bytearray, not NoneType

In [ ]:
doc_index = {d['id']: d for d in documents}

In [ ]:
final_results = []

for doc_id, questions in parsed_resulst.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [ ]:
df.to_csv('ground-truth-data.csv', index=False)

In [ ]:
!head ground-truth-data.csv